# advent of code 2024 - [day 15](https://adventofcode.com/2024/day/15)

### Parsing

In [ ]:

def gen_map(file='input.txt'):
    file = open(file, 'r')
    for row, line in enumerate(file):
        line = line.strip()
        if line == '':
            break
        for col, val in enumerate(tuple(line)):
            yield row, col, val
        

def gen_moves(file='input.txt'):
    file = open(file, 'r')
    state = "wait"
    new_state = state
    for ix, line in enumerate(file):
        line = line.strip()
        state = new_state
        if line == '':
            new_state = 'go'
        if state == 'wait':
            continue
        for move in tuple(line):
            yield move


def gen_map_part2(file='input.txt'):
    file = open(file, 'r')
    for row, line in enumerate(file):
        line = line.strip()
        if line == '':
            break
        for col, val in enumerate(tuple(line)):
            yield row, 2*col, {'#':'#', '@':'@', 'O':'[', '.':'.'}[val]
            yield row, 2*col+1, {'#':'#', '@':'.', 'O':']', '.':'.'}[val]

In [ ]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']


In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

import pandas as pd

In [ ]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

## Part 1

In [ ]:
def ingest(filename):
    clean()

    tiles = [{"row": row, "col": col, "val": val} for row, col, val in gen_map(filename)]
    moves = [{"ix": ix, "move": move} for ix, move in enumerate(gen_moves(filename))]

    query_ingest_tiles = """
    UNWIND $tiles AS tile
    CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
    """
    query_ingest_moves ="""
    UNWIND $moves AS move
    CREATE (:Move {ix: move.ix, move: move.move})
    """
    for q in [query_ingest_tiles, query_ingest_moves]:
        gds.run_cypher(q, {"tiles":tiles, "moves": moves})

    gds.run_cypher('CREATE INDEX tile_row IF NOT EXISTS FOR (r:Tile) ON (r.row)')
    gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col)')
    gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (r:Tile) ON (r.val)')
    gds.run_cypher('CREATE INDEX tile_col_row IF NOT EXISTS FOR (r:Tile) ON (r.col, r.row)')
    gds.run_cypher('CREATE INDEX tile_all IF NOT EXISTS FOR (r:Tile) ON (r.col, r.row, r.val)')
    gds.run_cypher('CREATE INDEX next_t_move FOR ()-[r:NEXT_TO]-() ON (r.move)')


    gds.run_cypher("""
    MATCH (t:Tile)
    WITH t.row AS row_num, t ORDER BY t.col
    WITH row_num, collect(t) AS row
    CALL apoc.nodes.link(row, 'EAST')
    """)

    gds.run_cypher("""
    MATCH (t:Tile)
    WITH t.col AS col_num, t ORDER BY t.row
    WITH col_num, collect(t) AS col
    CALL apoc.nodes.link(col, 'SOUTH')
    """)

    gds.run_cypher("""
    MATCH (a)-[r:EAST]->(b)
    MERGE (a)-[:NEXT_TO {move: '>'}]->(b)
    MERGE (b)-[:NEXT_TO {move: '<'}]->(a)              
    """)

    gds.run_cypher("""
    MATCH (a)-[r:SOUTH]->(b)
    MERGE (a)-[:NEXT_TO {move: 'v'}]->(b)
    MERGE (b)-[:NEXT_TO {move: '^'}]->(a)
    """)

    gds.run_cypher("""
    MATCH (m:Move)
    WITH m ORDER BY m.ix
    WITH collect(m) AS seq
    CALL apoc.nodes.link(seq, 'NEXT_MOVE')
    """)


In [ ]:
def print_map():
    print(gds.run_cypher("""
    MATCH (t:Tile)
    ORDER BY  t.col
    WITH t.row AS row_num, collect(t) AS tile_row
    WITH row_num, apoc.text.join([x IN tile_row | x.val],'') AS row
    ORDER BY row_num
    WITH collect(row) AS rows
    WITH apoc.text.join(rows, '\n') AS map
    RETURN map""").iloc[0]['map'])

    print(gds.run_cypher("""
    OPTIONAL MATCH (m:Move&!Processed)
    WHERE NOT EXISTS {(:Move&!Processed)-[:NEXT_MOVE]->(m)}
    RETURN m.move AS move""").iloc[0]['move'])


In [ ]:
filename= "input.txt"
ingest(filename)

In [ ]:
def infer (rules, params={}, verbose=False):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
                if verbose:
                    print_map()
            any_new_update = result.consume().counters._contains_updates
            any_update = any_update or any_new_update
        if not any_update:
            break

In [ ]:
move_query = """
MATCH (m:Move&!Processed) WHERE NOT EXISTS {(:Move&!Processed)-[:NEXT_MOVE]->(m)}
SET m:Processed
WITH m
MATCH (robot:Tile {val:'@'})
MATCH path = (robot)(
(xi WHERE xi.val IN ['O', '@'])-[:NEXT_TO {move: m.move}]->(y)
)*(free:Tile {val: '.'})
WITH m, robot, xi[1] AS new_robot, xi[-1] AS last_occupied, free, path
SET free.val = last_occupied.val, new_robot.val = '@', robot.val = '.'
"""

In [ ]:
infer([move_query])

In [ ]:
gds.run_cypher("""
MATCH (O:Tile {val:'O'})
WITH collect(O.row * 100 + O.col) AS gpss
RETURN reduce (acc = 0, gps IN gpss | acc+gps) AS part1
""")

# Part 2

In [ ]:
def infer (rules, params={}, verbose=False):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            if verbose:
                print(rule)
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
                if verbose:
                    print(result)
            any_new_update = result.consume().counters._contains_updates
            if verbose and any_new_update:
                print_map()
            any_update = any_update or any_new_update
        if not any_update:
            break

In [ ]:
def ingest_2(filename):
    clean()

    tiles = [{"row": row, "col": col, "val": val} for row, col, val in gen_map_part2(filename)]
    moves = [{"ix": ix, "move": move} for ix, move in enumerate(gen_moves(filename))]

    query_ingest_tiles = """
    UNWIND $tiles AS tile
    CREATE (:Tile {row:tile.row, col:tile.col, val:tile.val} )
    """
    query_ingest_moves ="""
    UNWIND $moves AS move
    CREATE (:Move {ix: move.ix, move: move.move})
    """
    for q in [query_ingest_tiles, query_ingest_moves]:
        gds.run_cypher(q, {"tiles":tiles, "moves": moves})

    gds.run_cypher('CREATE INDEX tile_row IF NOT EXISTS FOR (r:Tile) ON (r.row)')
    gds.run_cypher('CREATE INDEX tile_col IF NOT EXISTS FOR (r:Tile) ON (r.col)')
    gds.run_cypher('CREATE INDEX tile_val IF NOT EXISTS FOR (r:Tile) ON (r.val)')
    gds.run_cypher('CREATE INDEX tile_col_row IF NOT EXISTS FOR (r:Tile) ON (r.col, r.row)')
    gds.run_cypher('CREATE INDEX tile_all IF NOT EXISTS FOR (r:Tile) ON (r.col, r.row, r.val)')
    gds.run_cypher('CREATE INDEX next_t_move FOR ()-[r:NEXT_TO]-() ON (r.move)')


    gds.run_cypher("""
    MATCH (t:Tile)
    WITH t.row AS row_num, t ORDER BY t.col
    WITH row_num, collect(t) AS row
    CALL apoc.nodes.link(row, 'EAST')
    """)

    gds.run_cypher("""
    MATCH (t:Tile)
    WITH t.col AS col_num, t ORDER BY t.row
    WITH col_num, collect(t) AS col
    CALL apoc.nodes.link(col, 'SOUTH')
    """)

    gds.run_cypher("""
    MATCH (a)-[r:EAST]->(b)
    MERGE (a)-[:NEXT_TO {move: '>'}]->(b)
    MERGE (b)-[:NEXT_TO {move: '<'}]->(a)              
    """)

    gds.run_cypher("""
    MATCH (a)-[r:SOUTH]->(b)
    MERGE (a)-[:NEXT_TO {move: 'v'}]->(b)
    MERGE (b)-[:NEXT_TO {move: '^'}]->(a)
    """)

    gds.run_cypher("""
    MATCH (m:Move)
    WITH m ORDER BY m.ix
    WITH collect(m) AS seq
    CALL apoc.nodes.link(seq, 'NEXT_MOVE')
    """)


In [ ]:
def print_map():
    print(gds.run_cypher("""
    MATCH (t:Tile)
    ORDER BY  t.col
    WITH t.row AS row_num, collect(t) AS tile_row
    WITH row_num, apoc.text.join([x IN tile_row | CASE x:Movable WHEN TRUE THEN 'O' ELSE x.val END],'') AS row
    ORDER BY row_num
    WITH collect(row) AS rows
    WITH apoc.text.join(rows, '\n') AS map
    RETURN map""").iloc[0]['map'])

    print(gds.run_cypher("""
    OPTIONAL MATCH (m:Move&Processed)
    WHERE size(labels(m)) > 2
    RETURN m.move AS move, labels(m) AS state""").iloc[0][['move','state']])

    print(gds.run_cypher("""
    MATCH (m:Moveable)
    RETURN count(m) AS moveables""").iloc[0][['moveables']])

In [ ]:
filename= "input.txt"
ingest_2(filename)

In [ ]:
queries = ["""
MATCH (m:Move&!Processed)
WHERE NOT EXISTS {(:Updated|Moveable|CleanOrder|FreeOrder|MovingOrder|Current)}
AND NOT EXISTS {(:Move&!Processed)-[:NEXT_MOVE]->(m)}
WITH m
SET m:Current, m:Processed
""",
"""
MATCH (m:Move&Current)
MATCH (robot:Tile {val:'@'})
SET robot:Moveable
""",
"""
MATCH (m:Move&Current)
MATCH (:Moveable)-[:NEXT_TO {move:m.move}]->(pushed WHERE pushed.val in ['[',']'])
SET pushed:Moveable
WITH pushed
OPTIONAL CALL (pushed) {
    MATCH (pushed {val:'['})-[:EAST]->(same_box)
    SET same_box:Moveable
}
WITH pushed
OPTIONAL CALL (pushed) {
    MATCH (same_box)-[:EAST]->(pushed {val:']'})
    SET same_box:Moveable
}
""",
"""
MATCH (m:Move&Current)
WHERE EXISTS {(:Moveable)-[:NEXT_TO {move:m.move}]->(pushed WHERE pushed.val = '#')}
REMOVE m:Current
WITH m
MATCH (mvbl:Moveable)
REMOVE mvbl:Moveable
""",
"""
MATCH (m:Move&Current)
WHERE NOT EXISTS {(:Moveable)-[:NEXT_TO {move:m.move}]->(pushed:!Moveable WHERE pushed.val IN ['#', '[', ']'])}
REMOVE m:Current
SET m:MovingOrder
""",
"""
MATCH (m:Move&MovingOrder)
REMOVE m:MovingOrder
SET m:FreeOrder
WITH m
MATCH (source:Moveable)-[:NEXT_TO {move:m.move}]->(target)
WITH source.val AS new_val, target 
SET target.val = new_val, target:Updated
""",
"""
MATCH (m:Move:FreeOrder)
REMOVE m:FreeOrder
SET m:CleanOrder
WITH m
MATCH (source:Moveable&!Updated)
SET source.val = '.'
""",
"""
MATCH (m:Move:CleanOrder)
WITH m, ['Updated','Moveable','CleanOrder','FreeOrder','MovingOrder','Current'] AS labels
UNWIND labels AS l
MATCH (n:$(l))
REMOVE n:$(l)
"""]

#infer(queries, {}, True) #verbose
infer(queries, {})

In [165]:
gds.run_cypher("""
MATCH (O:Tile {val:'['})
WITH collect(O.row * 100 + O.col) AS gpss
RETURN reduce (acc = 0, gps IN gpss | acc+gps) AS part1
""")

,part1
0,1467145
